## 如何使用Python处理Missing Data

现实世界的数据中常常包含缺失的数据。原因很多，比如观察结果没有记录，或数据损坏。处理缺失的数据很重要，因为许多机器学习算法不支持具有缺失值的数据库。

本教程将讨论如何使用Python处理缺失的数据来进行机器学习。

### 您将了解到：
- 如何在数据集中标记无效或损坏的值。
- 如何从数据集中删除缺失数据的行。
- 如何使用数据集的均值估算缺失值。

**注意**：文章中的例子前提是安装了Python 2或3，Pandas，NumPy和Scikit-Learn的；特别是scikit-learn版本0.18或更高版本。

本教程分为6部分：

 - **皮马印第安人糖尿病数据集：**我们在哪里查看已知缺失值的数据集。
 - **标记缺失值：**我们学习如何在数据集中标记缺失值。
 - **缺少值导致的问题：** 机器学习算法如果包含缺少值，将会如何失败。
 - **删除缺少值的行：** 如何删除包含缺失值的行。
 - **估算缺失值：**我们用合理的值替换缺失值。
 - **允许缺失值的算法：**哪些算法允许缺失值。

首先，我们来看看缺少值的示例数据集。

###                    **皮马印第安人糖尿病数据集**

皮马印第安人糖尿病数据集根据现有的医疗信息预测5年内皮马印第安人糖尿病发作的概率。

这是两类(2-class)分组问题，每组的观察标本量不同。共有768个观测值，8个输入变量和1个输出变量。变量名称如下：

  0. 怀孕次数。
  1. 口服葡萄糖耐量试验中血浆葡萄糖浓度为2小时。
  2. 舒张压（mm Hg）。
  3. 三头肌组织褶厚度（mm）。
  4. 2小时血清胰岛素（μU/ ml）。
  5. 体重指数（kg/ （身高(m)）^ 2）。
  6. 糖尿病系统功能。
  7. 年龄（岁）。
  8. 类变量（pos或neg）。

预测的标准是大约65％的分类精准度。最好结果的分类精度约为77％。


已知此数据集具有缺失值。

我们可以通过这些列的意义和这些度量值是否可能为空来证实这一点。

### 标记缺失值

我们可以使用图表（plots）和汇总统计信息来帮助识别缺失或损坏的数据。

我们可以将数据集加载为Pandas DataFrame，并查看每个属性的统计摘要。

 - 在PDAS平台中， 只需要将数据集上传到项目中， 就可以通过下述方式将数据加载到Pandas Dataframe
 
运行代码输出结果

In [55]:
%config InlineBackend.figure_format='retina'
%matplotlibt inline 

import pandas as pd, numpy as np
from palan import dataset

diabetes_dataset = dataset.Dataset("diabetes")
diabetes = diabetes_dataset.get_dataframe()

print(diabetes.shape)
diabetes.describe()

Populating the interactive namespace from numpy and matplotlib
['pregnant', 'glucose', 'pressure', 'triceps', 'insulin', 'mass', 'pedigree', 'age', 'diabetes']
(768, 9)


,pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000


这种方法非常有用。我们可以看到数据集各个列值的计数。在某些列上，missing data是不计入，表示为无效值或缺失值。因此， 可以看到 glucose, pressure, triceps, insulin, mass列的计数小于数据集的行数。

具体来说，以下列具有无效的空值：

  1. 血浆葡萄糖浓度
  2. 舒张压
  3. 三头肌组织褶厚度
  4. 2小时血清胰岛素
  5. 体重指数

让我们来看看原始数据，这个例子显示出前20行的数据。

观察运行结果， 可以很清楚的看到glucose, pressure, triceps, insulin, mass列中包含的空值NaN。

 - 在Python中，特别是Pandas，NumPy和Scikit-Learn，我们将缺失值标记为NaN。在sum，count等操作中，NaN值的值将被忽略。
 
另外，发现二分变量 diabetes列为字符型， 为了便于后续计算， 先将其转换为数值型。代码如下：

In [56]:
# 打印数据集diabetes前20行
diabetes.head(20)  # diabetes 包含字符串

,pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,pos
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,neg
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,pos
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,neg
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,pos
5,5,116.0,74.0,NaN,NaN,25.6,0.201,30,neg
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,pos
7,10,115.0,NaN,NaN,NaN,35.3,0.134,29,neg
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53,pos
9,8,125.0,96.0,NaN,NaN,NaN,0.232,54,pos


In [57]:
# 转换 diabetes 列
# 将 pos(postive) 重编码为1
# 将 neg(negative) 重编码为0

diabetes.diabetes = np.where(diabetes.diabetes.values=='pos', 1, 0)
print('\n\n转换diabetes列后的数据集\n\n')
diabetes.head(20)



转换diabetes列后的数据集




,pregnant,glucose,pressure,triceps,insulin,mass,pedigree,age,diabetes
0,6,148.0,72.0,35.0,NaN,33.6,0.627,50,1
1,1,85.0,66.0,29.0,NaN,26.6,0.351,31,0
2,8,183.0,64.0,NaN,NaN,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
5,5,116.0,74.0,NaN,NaN,25.6,0.201,30,0
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26,1
7,10,115.0,NaN,NaN,NaN,35.3,0.134,29,0
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53,1
9,8,125.0,96.0,NaN,NaN,NaN,0.232,54,1


我们可以得到每列这些列中缺失值的数量。我们可以标记我们感兴趣的DataFrame的子集中的所有零值为真。然后，我们可以计算每列中missing data值的数量。

可以看到， triceps, insulin列中空值几乎占了一半。 而glucose, pressure, mass列的空值较少， 难怪在前二十行数据没有看到NaN。

这充分表明， 不同的列可能需要不同的策略来处理， 确保有足够的数据来训练模型。

在我们进一步处理缺失值之前，首先来看看数据集中缺失值可能会导致的问题。

In [58]:
diabetes.isnull().sum()

pregnant      0
glucose       5
pressure     35
triceps     227
insulin     374
mass         11
pedigree      0
age           0
diabetes      0
dtype: int64

### 缺少值导致问题

在数据集中缺少值可能会导致机器学习算法产生错误。

在本节中，我们将尝试评估带有缺失值的数据对线性判别分析（LDA）算法的影响。
 
当数据集中缺少值时，这种算法将无效。
 
下面的例子标记了数据集中的缺失值，就像我们在上一节中所做的那样，然后尝试使用3倍交叉验证来评估LDA，求平均精度。

In [59]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 数据拆分成输入和输出
values  = diabetes.values
X = values[:, 0:8]
y = values[:, 8]

# 在数据集上使用KFold交叉验证来评价线性判别分析模型
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result  = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

运行示例会导致错误。正如我们所料。我们无法在有缺失值的数据集上评估LDA和其他一些算法。

现在，我们来看看处理缺失值的方法。

### 删除缺失值的行

最简单的方法就是删除包含缺失值的行。
我们可以通过创建一个新的Pandas DataFrame，其中缺失值的行已经被删除。

Pandas提供了dropna() 函数，可用于删除列或缺少数据的行。我们可以使用dropna()来删除所有缺少数据的行，如下所示：

In [60]:
new_diabetes = diabetes.dropna()
new_diabetes.shape

(392, 9)

运行此示例，我们可以看到， 原始数据集中的768行大幅削减到了392行，所有都包含NaN的行都被删除了。

我们现在有一个数据集，我们可以使用它来评估LDA等对缺失值敏感的算法。

In [44]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 数据拆分成输入和输出
new_diabetes = diabetes.dropna()
values  = new_diabetes.values
X = values[:, 0:8]
y = values[:, 8]

# 在数据集上使用KFold交叉验证来评价线性判别分析模型
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result  = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.78582892934


例子运行成功，并能得到模型的准确性。

删除缺失值的行可能对不适用与某些预测建模问题，另一种方法则是估算丢失值。

### 估算缺失值

引用(imputing),指使用模型替换缺失值。
在替换缺少的值时可以考虑许多选择，例如：
 
    • 在域内具有含义的常量值，例如0，不同于其他所有值。
    • 来自另一个随机记录的值。
    • 该列的平均值，中值或模式值。
    • 由另一预测模型估计的值。
  

如果最终模型需要做预测，那么所有对于训练数据集进行的imputing都要应用到未来的新数据中。这会影响我们选择如何估算缺失值。
 
例如，如果您选择使用平均列值进行估算，这些平均值的列将需要存储到文件中，以备将来新数据含有缺失值时使用。
 
Pandas提供了fillna ( ) 函数来替换具有特定值的缺失值。例如，我们可以使用fillna ( )，平均值来替换每列的缺失值，如下所示：

In [61]:
import numpy as np

# 使用均值填充空值
new_diabetes = diabetes.fillna(diabetes.mean())

# 统计每列的空值数量
new_diabetes.isnull().sum()

pregnant    0
glucose     0
pressure    0
triceps     0
insulin     0
mass        0
pedigree    0
age         0
diabetes    0
dtype: int64

运行每列中缺少值的计数，显示缺失值为零。

scikit学习库提供可用于替换缺失值的Imputer ( ) 预处理类。

这是一个很灵活的类，允许指定要替换的值（可以是NaN以外的）和用于替换它的东西（如平均值，中值或众数）。Imputer类直接在NumPy数组而不是DataFrame上运行。

下面的示例使用Imputer类平均值替换每列的缺失值，然后得到转换矩阵中的NaN值的计数。

In [62]:
from sklearn.preprocessing import Imputer
import numpy as np

# 用均值来填充空值
values = diabetes.values
imputer = Imputer()
transformed_values = imputer.fit_transform(values)
# 统计每列空值的数量
print(np.isnan(transformed_values).sum())

0


运行示例显示所有NaN值成功替换。

无论哪种情况，我们都可以对缺失值敏感的算法（如LDA）使用转化后的数据集进行训练 。

下面的例子显示了在Imputer转换数据集中训练LDA算法

In [47]:
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 拆分数据集为输入和输出
values = diabetes.values
X = values[:, 1:8]
y = values[:, 8]

# 用列均值填充空值
imputer = Imputer()
transformed_X = imputer.fit_transform(X)

# 使用KFold交叉验证来评估线性判别模型
model = LinearDiscriminantAnalysis()
kfold = KFold(n_splits=3, random_state=7)
result = cross_val_score(model, transformed_X, y, cv=kfold, scoring='accuracy')
print(result.mean())

0.764322916667


得到转换数据集上LDA的准确性。
        
     0.764322916667
      


尝试用其他值替换缺少的值，并查看是否可以提升模型的表现。
也许缺少值在数据中是有意义的。
 
接下来，我们将使用将缺失值视为另一个值的做法。

### 支持缺失值的算法

当缺少数据时，并不是所有的算法都会失效。

有一些可以灵活对待缺失值的算法，例如k-Nearest Neighbors，当值缺失时，它可以将其不计入距离测量。
 
另一些算法，例如分类和回归树，可以在构建预测模型时将缺失值看作唯一且不同的值。

遗憾的是，决策树和k-Nearest Neighbors对于缺失值并不友好。
 
不管怎样，如果你考虑使用其他算法（如xgboost）或开发自己的执行，这依然是一个选择。